In [11]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR, NuSVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [ ]:
# First we run the sript to generate all the variables we'll use

# Generate train file
#%run ../scripts/create_variables.py -f ../data/train.csv -c excerpt -nc cleaned_text -nf processed_analysed_train.csv
# Generate test file
#%run ../scripts/create_variables.py -f ../data/test.csv -c excerpt -nc cleaned_text -nf processed_analysed_test.csv


In [2]:
# Next we read the data
train_df = pd.read_csv('../data/outputs/processed_analysed_train.csv')
test_df = pd.read_csv('../data/outputs/processed_analysed_test.csv')

In [3]:
# Create the variables to predict and to train with
drop_feat = ['excerpt', 'cleaned_text', 'id', 'standard_error', 'target', 'url_legal', 'license']
X = train_df.drop(drop_feat, axis=1)
y = train_df['target']
X.head()

,friend,alway,light,you,name,end,carri,set,though,need,...,sentence_count,sentence_score,rd_automatedindex,rd_fogscale,rd_colemanliau,rd_flesch_ease,rd_linearwrite,rd_fleschkincaid_grade,rd_dalechall,rd_consensus
0,0.0,0.0,0.076923,0.0,0.0,0.75,0.0,0.0,0.0,0.0,...,11,1.3431,8.3,8.31,8.06,80.31,9.000000,6.1,6.65,9.0
1,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,12,1.5504,7.2,7.53,6.78,82.54,7.285714,5.2,5.92,8.0
2,0.0,0.0,0.000000,0.2,0.0,0.00,0.0,0.0,0.0,0.0,...,8,1.0710,10.1,10.49,7.20,75.74,14.750000,7.9,6.29,8.0
3,0.0,0.0,0.000000,0.0,0.0,0.25,0.0,0.0,0.0,0.0,...,5,0.6693,16.4,13.61,8.54,72.02,12.500000,11.4,6.61,7.0
4,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,5,0.8666,11.8,11.76,4.83,75.47,13.500000,10.0,1.57,12.0


## SVR models

In [ ]:
# We leave out 'linear' and 'sigmoid' due to their bad results
svr_kernels = ['poly', 'rbf']
#gamma = np.arange(0.1, 1, 0.4)
gam=0.1

svr_pred =\
    [SVR(kernel=ker, C=100, gamma=gam, degree=3, epsilon=.1, coef0=1).fit(X, y).predict(X)\
     for ker in svr_kernels]

svr_acc = [mean_squared_error(y, y_pred) for y_pred in svr_pred]

#display([(ker, gam, acc) for ker in svr_kernels for gam in gamma for acc in svr_acc\
#         if acc <= 0.01])

for ker, acc in list(zip(svr_kernels, svr_acc)):
    print(ker + ": " + str(acc))


## NuSVR models

In [ ]:
nusvr_kernels = ['linear', 'poly', 'rbf', 'sigmoid']

nusvr_pred =\
    [NuSVR(kernel=ker, C=100, gamma=0.1, degree=3, nu=.1, coef0=1).fit(X, y).predict(X)\
     for ker in nusvr_kernels]

nusvr_acc = [mean_squared_error(y, y_pred) for y_pred in nusvr_pred]

for ker, acc in list(zip(nusvr_kernels, nusvr_acc)):
    print(ker + ": " + str(acc))

## XGBoost model

In [4]:
# First let's separate the training data into train and test data
data_train, data_val, target_train, target_val = \
    train_test_split(train_df, train_df["target"], test_size=0.3, random_state=5)

# As before, drop irrelevant features or features that we do not need
X_train = data_train.drop(drop_feat, axis=1)
X_val = data_val.drop(drop_feat, axis=1)

In [5]:
xgbreg = XGBRegressor()
xgbreg.fit(X_train, target_train)

kfold = KFold(n_splits=5, random_state=7, shuffle=True)
results = cross_val_score(xgbreg, X_train, target_train, cv=kfold)

y_test_pred = xgbreg.predict(X_val)
mse = mean_squared_error(y_test_pred, target_val)
print("Mean Squared Error: " + str(mse))

Mean Squared Error: 0.6471750555617857


## Random Forest

In [6]:
train = train_df.drop(['id', 'url_legal', 'license', 'excerpt', 'cleaned_text'], axis=1)
train.head()
# Remove the targets from the set
# Convert pandas df to np arrays
targets = np.array(train['target'])
feats = train.drop(['target'], axis=1)
features_list = list(feats.columns)
features = np.array(feats)
# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=20)
#### NOTE! I'm setting random_state to 20 so we get the same results every time we run the split.
#### Results are reproducible, but maybe this is not the best approach


In [7]:
# Sanity check: we expect the training features number of columns to match the testing feature number of columns 
# and the number of rows to match for the respective training and testing features and the labels
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (2267, 515)
Training Labels Shape: (2267,)
Testing Features Shape: (567, 515)
Testing Labels Shape: (567,)


In [8]:
# Instantiate model with 1000 decision trees
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 20)
#### NOTE! random_state again

# Train the model on training data
rf.fit(X_train, y_train)

pred = rf.predict(X_test)

In [12]:
# Compute performance
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, pred)))

Mean Absolute Error (MAE): 0.5254135828139506
Mean Squared Error (MSE): 0.5259818403639591
Root Mean Squared Error (RMSE): 0.7252460550488773


In [13]:
### MODEL ANALYSIS

# Get numerical feature relevance
importances = list(rf.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True) 
for pair in feature_importances:
    print('Variable: {:20}{}'.format(*pair))
## All the word-variables related to the TF-IDF analysis don't seem to be relevant for the model.
## We might assume that we don't need information from isolated words.
## We can retest the model using only some of the readability measures.

Variable: rd_dalechall        0.36
Variable: standard_error      0.21
Variable: lexicon_count       0.02
Variable: rd_fogscale         0.02
Variable: rd_linearwrite      0.02
Variable: brain               0.01
Variable: punctuation_count   0.01
Variable: punctuation_score   0.01
Variable: lexicon_score       0.01
Variable: sentence_score      0.01
Variable: rd_automatedindex   0.01
Variable: rd_colemanliau      0.01
Variable: rd_flesch_ease      0.01
Variable: rd_fleschkincaid_grade0.01
Variable: friend              0.0
Variable: alway               0.0
Variable: light               0.0
Variable: you                 0.0
Variable: name                0.0
Variable: end                 0.0
Variable: carri               0.0
Variable: set                 0.0
Variable: though              0.0
Variable: need                0.0
Variable: that                0.0
Variable: right               0.0
Variable: night               0.0
Variable: young               0.0
Variable: quit                0.

In [14]:
features_list = ['rd_dalechall', 'standard_error', 'lexicon_count', 'punctuation_count', 'rd_flesch_ease', 'rd_colemanliau', 'rd_fogscale']
targets = np.array(train['target'])
features = np.array(train[features_list])
# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=20)
#### NOTE! I'm setting random_state to 20 so we get the same results every time we run the split.
#### Results are reproducible, but maybe this is not the best approach

In [15]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 20)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
# Compute performance
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, pred)))

Mean Absolute Error (MAE): 0.5460683148735698
Mean Squared Error (MSE): 0.5263316876956162
Root Mean Squared Error (RMSE): 0.7254872071205779


In [ ]:
## We get almost the same results with this version. However, the results are not yet optimal.
## Needs more fine tuning!

In [ ]:
# Visualization of a single tree
from sklearn.tree import export_graphviz
import pydot
from IPython.core.display import Image, display

tree = rf.estimators_[10]
export_graphviz(tree, out_file = 'tree.dot', feature_names = features_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('tree.dot')
graph.write_png('tree.png')
display(Image('tree.png', height=1700, width=29000, unconfined=True))

## GLM

In [17]:
import statsmodels.api as sm

features_list = ['rd_dalechall', 'standard_error', 'lexicon_count', 'punctuation_count', 'rd_flesch_ease', 'rd_colemanliau', 'rd_fogscale']
endog = np.array(train['target'])
exog = sm.add_constant(np.array(train[features_list]))

glmGauss = sm.GLM(endog, exog, family=sm.families.Gaussian()).fit()
glmGamma = sm.GLM(endog, exog, family=sm.families.Gamma()).fit()

print(glmGauss.summary())
print(glmGamma.summary())

## Both models exhibit a very poor performance. Thus, we might discard them.

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                 2834
Model:                            GLM   Df Residuals:                     2826
Model Family:                Gaussian   Df Model:                            7
Link Function:               identity   Scale:                         0.69140
Method:                          IRLS   Log-Likelihood:                -3494.3
Date:                Wed, 16 Jun 2021   Deviance:                       1953.9
Time:                        01:47:32   Pearson chi2:                 1.95e+03
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.7802      0.498      7.591      0.0

### Conclusions:

We conclude that GLM is not valid to predict readability complexity, at least with the data that we have. On the other hand, Random Forest looks like a better alternative to GLM. Although not optimal, the results seem quite reasonable. The model would benefit from finding a way to take advantage from TF-IDF. If we look at the MSE, the results exhibit by the Random Forest are a little bit better than the XGBoost ones.

## Future work

Since there is still plenty of time to the competition, a section on future work is in order.

We have tested more traditional methods of dealing with NLP, howerver, recent advances have created a family of models which we have barely touhed and could yield promissing results.

Such models are BERT (of which we have two simple scripts), RoBERTa, ALBERT... All based on BERT, which stands for Bidirectional Encoder Representations from Transformers.

Thus, the natural course of action to take now is to explore such rich family of models and see which fits best our purouses.